In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/PCB_QM

/content/gdrive/MyDrive/PCB_QM


In [3]:
import pickle

with open('test_pickle.pickle','rb') as file :
    pk_data = pickle.load(file)

In [5]:
pk_data.keys()

dict_keys(['img_list', 'label_list'])

In [7]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [8]:
from tensorflow.keras.layers import BatchNormalization,Dropout

In [9]:
from sklearn.model_selection import train_test_split

In [ ]:
class Model_Train:
    def __init__(self, data_pickle,img_list_col='img_list',prod_col='product_type',error_col='error_type', random_seed = 42):
        self.seed = random_seed
        tf.set_random_seed(self.seed)
        self.load_pickle(img_list_col,prod_col,error_col)

    def load_pickle(self,pickle_path,img_list_col,prod_col,error_col):
        with open(pickle_path,'rb') as file:
            pk_data = pickle.load(file)
            # pickle data를 numpy array 형태로 저장
            self.image_set = np.array(pk_data[img_list_col])
            self.prod_label_set = np.array(pk_data[prod_col])
            self.error_label_set = np.array(pk_data[error_col])

        self.img_shape = self.image_set[0].shape
        self.error_label_num = len(np.unique(self.error_label_set))
        self.class_dict = {v: k for k, v in enumerate(np.unique(self.error_label_set))}

    def gen_train_test_set(self,test_rate = 0.1):
        '''학습 및 테스트 데이터를 분류'''
        self.error_img_train, self.error_img_test, self.error_label_train, self.error_label_test = train_test_split(self.image_set,self.error_label_set,
                                                                                                  test_size = test_rate, random_state = self.seed,
                                                                                                  stratify= self.error_label_set)

    def model_generate(self,fliter_num:int = 32, conv_layer:int = 2, dense_unit:int = 128 , drop_rate:float = 0.2 ) :
        '''모델 생성'''
        # 모델 초기화
        model = Sequential()

        # 초기 레이어 생성
        model.add(Conv2D(fliter_num, (3, 3), activation='relu', input_shape=self.img_shape))
        model.add(Conv2D(fliter_num, (3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(drop_rate))

        try :
            # 레이어 추가
            for i in range(1, conv_layer)
                model.add(Conv2D(fliter_num * (i+1), (3, 3), activation='relu', input_shape=self.img_shape))
                model.add(Conv2D(fliter_num * (i+1), (3, 3), activation='relu'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D((2, 2)))
                model.add(Dropout(drop_rate))
        except Exception:
            print('conv_laber에 2 이상 값을 넣어야 레이어 추가가 됩니다.')

        # 전연결
        model.add(Flatten())
        model.add(Dense(dense_unit, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(drop_rate))
        model.add(Dense(self.error_label_num, activation='softmax'))

        # 모델 요약 출력 (선택 사항)
        model.summary()

        model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

        self.model = model

    def train_basic(self,test_rate = 0.1, batch_size = 16, epochs = 5):
        '''테스트 데이터를 분류하고 에러 학습'''
        self.gen_train_test_set(test_rate)
        self.history = self.model.fit(self.error_train_img, self.error_label_train, batch_size = batch_size, epochs = epochs)
        print("학습이 끝났습니다. history를 통해 학습 과정을 확인할 수 있습니다.")

    def validation(self):
        '''테스트 데이터 정확도 확인'''
        # 전체 정확도 평가
        test_loss, test_accuracy = self.model.evaluate(self.error_test_img, self.error_label_test)
        print(f'Test Accuracy: {test_accuracy:.2f}\n\n')

        # 예측
        y_pred = self.model.predict(self.error_test_img)
        y_pred_classes = np.argmax(y_pred, axis=1)
        # 예측 결과와 원래 라벨 출력
        predicted_labels = [self.class_dict[k] for k in y_pred_classes]

        cnt = 0
        for real_label, pred_label in zip(self.error_label_test, predicted_labels):
            print(f"index : {cnt} --- 정답 라벨 : {real_label} ---> 예측 라벨 : {pred_label}")
            cnt+=1
